<a href="https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/MNIST-Augmentaion/MNET_Noise_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# -*- coding: utf-8 -*-
"""M-Net_1

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/Protocol/M_Net_1.ipynb
"""

# from m_net_1 import H_test, X_pred
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from tensorflow.keras.datasets import fashion_mnist, mnist

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, FunctionTransformer

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

# from google.colab import drive
# drive.mount('/content/drive')

def main(X, X_T, y_train, y_test, s_msg = " ", n_hidden=23):

    rs = 23  # fester Random Seed
    np.random.seed(rs)

    results, params = set_params_results(n_hidden)
    results['MSG'] = s_msg


    """Create Hidden Activations"""
    H = np.random.rand(len(y_train), params['N_HIDDEN'])
    print(H.shape)
    print(H)

    """1st Forward Learn"""
    W, X_pred = forward_learn(H, X)
    results['MAE_1ST_FL'] = round(mean_absolute_error(X_pred, X), 3)
    results['MSE_1ST_FL'] = round(mean_squared_error(X_pred, X), 3)
    results['R2_1ST_FL'] = round(r2_score(X_pred, X, multioutput="variance_weighted"), 3)

    """1st Back Activations"""
    H, XT_pred = back_activation(W, X)
    XT = X.transpose()
    results['MAE_1ST_BA'] = round(mean_absolute_error(XT_pred, XT), 3)
    results['MSE_1ST_BA'] = round(mean_squared_error(XT_pred, XT), 3)
    results['R2_1ST_BA'] = round(r2_score(XT_pred, XT, multioutput="variance_weighted"), 3)

    """2nd Forward learn - result are Coherent Weights"""
    W, X_pred = forward_learn(H, X)
    results['MAE_2ND_FL'] = round(mean_absolute_error(X_pred, X), 3)
    results['MSE_2ND_FL'] = round(mean_squared_error(X_pred, X), 3)
    results['R2_2ND_FL'] = round(r2_score(X_pred, X, multioutput="variance_weighted"), 3)


    """2nd Back Activation - final representation of X_train in Hidden Activations"""
    H, XT_pred = back_activation(W, X)
    XT = X.transpose()
    results['MAE_2ND_BA'] = round(mean_absolute_error(XT_pred, XT), 3)
    results['MSE_2ND_BA'] = round(mean_squared_error(XT_pred, XT), 3)
    results['R2_2ND_BA'] = round(r2_score(XT_pred, XT, multioutput="variance_weighted"), 3)

    """Back Activation of Test Data"""
    H_test, XT_pred = back_activation(W, X_T)
    XT = X_T.transpose()
    results['MAE_TEST_BA'] = round(mean_absolute_error(XT_pred, XT), 3)
    results['MSE_TEST_BA'] = round(mean_squared_error(XT_pred, XT), 3)
    results['R2_TEST_BA'] = round(r2_score(XT_pred, XT, multioutput="variance_weighted"), 3)

    y_pred = knn_test(H_test, H, y_train)
    
    # Making the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    results['ACCURACY_SCORE'] = accuracy_score(y_test, y_pred)
    results['DICT_REPORT'] = str(classification_report(y_test, y_pred, output_dict=True))

    print(accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    df_results = pd.DataFrame([results])
    prot_row(df_results)

    return

"""Own Funs"""

def mnist_augmentation_noise(train_images, train_labels):

    noise_matrix = np.random.random((28,28))
    mnist_noise_matrix = noise_matrix * 255
    new_train_data = (train_images + mnist_noise_matrix) / 2
   
    extended_train_images = train_images
    extended_train_labels = train_labels

    print(extended_train_labels.shape)

    print ("original data:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(train_images.shape, np.mean(train_images), np.std(train_images)))

    print ("augmentated data:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(extended_train_images.shape, np.mean(extended_train_images), np.std(extended_train_images)))

    return extended_train_images, extended_train_labels


def mnist_augmentation_shift(train_images, train_labels):

    shift_train_data = np.roll(train_images,1)
    extended_train_images = np.append(train_images, shift_train_data, axis=0)
    extended_train_labels = np.append(train_labels, train_labels, axis=0)
 
    shift_train_data = np.roll(train_images,-1)
    extended_train_images = np.append(extended_train_images, shift_train_data, axis=0)
    extended_train_labels = np.append(extended_train_labels, train_labels, axis=0)

    shift_train_data = np.roll(train_images,28)
    extended_train_images = np.append(extended_train_images, shift_train_data, axis=0)
    extended_train_labels = np.append(extended_train_labels, train_labels, axis=0)
 
    shift_train_data = np.roll(train_images,-28)
    extended_train_images = np.append(extended_train_images, shift_train_data, axis=0)
    extended_train_labels = np.append(extended_train_labels, train_labels, axis=0)

    print(extended_train_labels.shape)

    print ("original data:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(train_images.shape, np.mean(train_images), np.std(train_images)))

    print ("augmentated data:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(extended_train_images.shape, np.mean(extended_train_images), np.std(extended_train_images)))

    return extended_train_images, extended_train_labels

def preprocess(X_train, X_test):

    # NEW Norm

    X = X_train.reshape(len(X_train), -1).astype('int')
    X = pd.DataFrame(X)
    X = X.applymap(ifelse)
    X = X.apply(rms_norm, axis=1)
    X = np.array(X)

    print ("Originaldaten:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_train.shape, np.mean(X_train), np.std(X_train)))

    print ("Vorbereitete Daten:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

    X_T = X_test.reshape(len(X_test), -1).astype('int')
    X_T = pd.DataFrame(X_T)
    X_T = X_T.applymap(ifelse)
    X_T = X_T.apply(rms_norm, axis=1)
    X_T = np.array(X_T)

    print ("Originaldaten:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_test.shape, np.mean(X_test), np.std(X_test)))

    print ("Vorbereitete Daten:")
    print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_T.shape, np.mean(X_T), np.std(X_T)))

    return X, X_T

def ifelse(a):
    if a == 0:
        return 1
    else:
        return a


def rms_norm(v_x):
    n_rms = np.sqrt(np.sum(v_x ** 2)/(len(v_x) - 1))
    return v_x/n_rms


def prot_row(df_results):
    """
    make a new row in protokoll

    Parameters
    ----------
    outcome : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    # Colab file dir also on local 
    prot_file = '/content/sample_data/M-Net_Protocol.xlsx'

    # local file dir old
    # prot_file = 'M-Net_Protocol.xlsx'
    if os.path.isfile(prot_file):
        df_prot = pd.read_excel (prot_file)
        df_prot = df_prot.append(df_results, 
                          ignore_index=True)
    else:
        df_prot = df_results

    df_prot.to_excel(prot_file, index=False)
        
    return None

"""Params and Results"""
def set_params_results(n_hidden):

  params = {'N_HIDDEN': n_hidden}
  results = {'TIMESTAMP': datetime.now(),
            'NUMBER_OF_HIDDEN': params['N_HIDDEN'],
            'ACCURACY_SCORE': 0, 
            'MAE_1ST_FL': 0,
            'MSE_1ST_FL': 0,
            'R2_1ST_FL': 0,
            'MAE_1ST_BA': 0,
            'MSE_1ST_BA': 0,
            'R2_1ST_BA': 0,
            'MAE_2ND_FL': 0,
            'MSE_2ND_FL': 0,
            'R2_2ND_FL': 0,
            'MAE_2ND_BA': 0,
            'MSE_2ND_BA': 0,
            'R2_2ND_BA': 0,
            'MAE_TEST_BA': 0,
            'MSE_TEST_BA': 0,
            'R2_TEST_BA': 0,
            'DICT_REPORT': '',
            'MSG':'M-Net on python'}

  return(results, params)


def forward_learn(H, X):

    reg = LinearRegression().fit(H, X)
    print(reg.score(H, X))
    X_pred = reg.predict(H)

    print(f'MAE {mean_absolute_error(X_pred, X)}')
    print(f'MSE {mean_squared_error(X_pred, X)}')
    print(f'R2 {r2_score(X_pred, X)}')
    print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
    print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

    W = reg.coef_
    print(W.shape)
    WI = reg.intercept_
    print(WI.shape)

    return W, X_pred


def back_activation(W, X):

    XT = X.transpose()
    print(f'shape of X_train {X.shape} and X_train transposed {XT.shape}')

    reg = LinearRegression().fit(W, XT)
    print(reg.score(W, XT))
    XT_pred = reg.predict(W)

    print(f'MAE {mean_absolute_error(XT_pred, XT)}')
    print(f'MSE {mean_squared_error(XT_pred, XT)}')
    print(f'R2 {r2_score(XT_pred, XT)}')
    print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
    print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

    H_new = reg.coef_
    print(H_new.shape)
    HI = reg.intercept_
    print(HI.shape)

    """Hidden Norm"""
    H_new = pd.DataFrame(H_new)
    H_new = H_new.apply(rms_norm, axis=1)
    H_new = np.array(H_new)

    return H_new, XT_pred


"""KNN test with Hidden Activations"""

def multi_knn_test(H_test, H, y_train,):
    """ Majority of 3 kNN with different k - Weigthed kNN has similar results with one k """

    # k1
    neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, weights='distance')
    neigh.fit(H, y_train)
    # Predicting the Test set results
    y_pred_1 = neigh.predict(H_test)
    # k2
    neigh = KNeighborsClassifier(n_neighbors=3, n_jobs=-1, weights='distance')
    neigh.fit(H, y_train)
    # Predicting the Test set results
    y_pred_2 = neigh.predict(H_test)
    # k3
    neigh = KNeighborsClassifier(n_neighbors=1, n_jobs=-1, weights='distance')
    neigh.fit(H, y_train)
    # Predicting the Test set results
    y_pred_3 = neigh.predict(H_test)

    y_pred_all = pd.DataFrame(y_pred_1)
    y_pred_all['2'] = y_pred_2
    y_pred_all['3'] = y_pred_3
    y_pred_all['majority'] = y_pred_all.mode(axis=1)[0]

    y_pred = y_pred_all['majority'].to_numpy()

    return y_pred

def knn_test(H_test, H, y_train,):

    neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, weights='distance')
    neigh.fit(H, y_train)
    # Predicting the Test set results
    y_pred = neigh.predict(H_test)

    return y_pred

def load_data(n_subset = False):

    # Fashion MNIST Daten laden, wir wollen nur die Trainingsdaten und verwerfen die Testdaten
    # (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    if n_subset:
        X_train = X_train[:n_subset]
        y_train = y_train[:n_subset]

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    return X_train, X_test, y_train, y_test


# Code starting point

if __name__ == '__main__':
    X_train, X_test, y_train, y_test = load_data()
    # X_train, X_test, y_train, y_test = load_data(1000)

    X_train, y_train = mnist_augmentation_shift(X_train, y_train)
    X_train, y_train = mnist_augmentation_noise(X_train, y_train)

    X, X_T = preprocess(X_train, X_test)
    for i in range(10, 60, 1):
        main(X, X_T, y_train, y_test, s_msg="k5 -shift + only - Noise Augmentation",n_hidden=i)


(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)
(300000,)
original data:
Shape: (60000, 28, 28), Mean: 33.318421, STD: 78.567490
augmentated data:
Shape: (300000, 28, 28), Mean: 33.318421, STD: 78.567490
(300000,)
original data:
Shape: (300000, 28, 28), Mean: 33.318421, STD: 78.567490
augmentated data:
Shape: (300000, 28, 28), Mean: 33.318421, STD: 78.567490
Originaldaten:
Shape: (300000, 28, 28), Mean: 33.318421, STD: 78.567490
Vorbereitete Daten:
Shape: (300000, 784), Mean: 0.396040, STD: 0.917538
Originaldaten:
Shape: (10000, 28, 28), Mean: 33.791224, STD: 79.172463
Vorbereitete Daten:
Shape: (10000, 784), Mean: 0.398067, STD: 0.916661
(300000, 10)
[[0.51729788 0.9469626  0.76545976 ... 0.39244247 0.61805235 0.41193009]
 [0.00246488 0.88403218 0.88494754 ... 0.06507544 0.29474446 0.28793444]
 [0.82246634 0.62618304 0.11047771 ... 0.34648944 0.86978508 0.42860181]
 ...
 [0.57993955 0.97995646 0.69451199 ... 0.93718931 0.68180991 0.29239845]
 [0.12149473 0.7072983  0.12076273 ... 0.

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


3.478201789576343e-05
MAE 0.44542311581906
MSE 0.6043586061517773
R2 -36878.63079917024
R2 vw -28748.488340532338
R2 ua -36878.63079917024
(784, 10)
(784,)
shape of X_train (300000, 784) and X_train transposed (784, 300000)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.1792703266073792
MAE 0.5810604616312589
MSE 0.6882793165872094
R2 -4.475850447937229
R2 vw -3.5781680042905495
R2 ua -4.475850447937229
(300000, 10)
(300000,)


...

0.7730709709885508
MAE 0.2034231066657882
MSE 0.1371512820670829
R2 -17.579548555766873
R2 vw 0.7064577024263788
R2 ua -17.579548555766873
(784, 59)
(784,)
shape of X_train (300000, 784) and X_train transposed (784, 300000)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.8310742979246993
MAE 0.25188445329700826
MSE 0.14166426601561685
R2 0.7898011630376726
R2 vw 0.7967381466408839
R2 ua 0.7898011630376726
(300000, 59)
(300000,)
shape of X_train (10000, 784) and X_train transposed (784, 10000)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.8445151859993161
MAE 0.24356723643168343
MSE 0.13014036442517862
R2 0.8110287465492635
R2 vw 0.8158886701170464
R2 ua 0.8110287465492635
(10000, 59)
(10000,)
[[ 974    0    1    0    0    1    3    1    0    0]
 [   0 1130    3    0    1    0    1    0    0    0]
 [   5    1 1010    1    2    0    0   11    2    0]
 [   1    2    2  981    0    4    0    6   11    3]
 [   0    2    0    0  956    0    5    2    3   14]
 [   2    0    1   12    1  858    8    1    8    1]
 [   5    2    0    0    2    1  947    0    1    0]
 [   0    8    7    0    2    0    0 1004    0    7]
 [   6    1    1    7    3    4    2    4  944    2]
 [   5    3    0    3    5    5    1    6    1  980]]
0.9784
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.98      1.00      0.99      1135
           2       0.99      0.98      0.98      1032
           3       0.98      0.97      0.97      1010
           4       0.98      0.